In [1]:
%pwd

'e:\\Vscode\\MLOPS-1\\Chest_Disease_Classification\\research'

In [4]:
import os
os.chdir("../")


In [5]:
%pwd

'e:\\Vscode\\MLOPS-1\\Chest_Disease_Classification'

In [6]:
from dataclasses import dataclass

from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    copy_of_model: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [3]:
create_directories(['CHEST/model/model.h5'])

[2024-03-16 17:28:39,196: INFO: common: directory:CHEST/model/model.h5 created successfully]


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self)->ModelTrainerConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data")
        create_directories([Path(training.root_dir)])

        training_config=ModelTrainerConfig(
            root_dir=Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            copy_of_model= Path(self.config.copy_of_model),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    




In [1]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import shutil

In [38]:
from scipy import interpolate


class Training:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20)
        
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
            )
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator
        
        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
            )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
        shutil.copy(self.config.trained_model_path,self.config.copy_of_model)

In [37]:
copy_of_model= 'Model'
os.makedirs(copy_of_model)
shutil.copy('artifacts/training/model.h5',copy_of_model)

'Model\\model.h5'

In [39]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-16 18:31:25,144: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-03-16 18:31:25,152: INFO: common: yaml file:params.yaml loaded successfully]
[2024-03-16 18:31:25,155: INFO: common: directory:artifacts created successfully]
[2024-03-16 18:31:25,158: INFO: common: directory:artifacts\training created successfully]


Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
17/17 [==============================] - 101s 6s/step - loss: 17.4924 - accuracy: 0.5019 - val_loss: 3.0584 - val_accuracy: 0.3906
